In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

In [2]:
def tps2_181(inputfile,outputfile):
    yourData = []
    with open(inputfile) as inputData:
        for line in inputData:
            try:
                yourData.append(json.loads(line.rstrip('\n')))
            except ValueError:
                print("Skipping invalid line {0}").format(repr(line))
    
    df = pd.DataFrame(yourData)
    
    temp=[]
    localtime = df.localtime
    for i in range(len(localtime)):
        temp.append(localtime[i][:-4])
    
    df['localtime'] = temp
    
    distinct_mac = df.ClientMacAddr.value_counts()
    temp = distinct_mac[(distinct_mac < 2000) & (distinct_mac > 50)]
    temp1 = temp.index.tolist()
    
    elapse = []
    for i in range(len(temp1)):
        a = df[df['ClientMacAddr'] == temp1[i]]
        d = a.localtime.max()
        f = a.localtime.min()
        fmt1 = '%Y-%m-%d %H:%M:%S.%f'
        fmt2 = '%Y-%m-%d %H:%M:%S'
        try:
            tstamp1 = datetime.strptime(d, fmt1)
            tstamp2 = datetime.strptime(f, fmt1)
            td = tstamp1 - tstamp2
            td_hours = td.total_seconds() / 3600
            elapse.append(td_hours)
        except:
            try:
                tstamp1 = datetime.strptime(d, fmt2)
                tstamp2 = datetime.strptime(f, fmt1)
                td = tstamp1 - tstamp2
                td_hours = td.total_seconds() / 3600
                elapse.append(td_hours)
            except:
                try:
                    tstamp1 = datetime.strptime(d, fmt1)
                    tstamp2 = datetime.strptime(f, fmt2)
                    td = tstamp1 - tstamp2
                    td_hours = td.total_seconds() / 3600
                    elapse.append(td_hours)
                except:
                    tstamp1 = datetime.strptime(d, fmt2)
                    tstamp2 = datetime.strptime(f, fmt2)
                    td = tstamp1 - tstamp2
                    td_hours = td.total_seconds() / 3600
                    elapse.append(td_hours)
                    
    df_time = pd.DataFrame()
    df_time['distinct_mac'] = temp1
    df_time['elapse_time'] = elapse
    
    temp3 = df_time[(df_time['elapse_time']<5) & (df_time['elapse_time']> 0.5)]
    mac = temp3.distinct_mac.tolist()
    tps2_181 = df[df['ClientMacAddr'].isin(mac)]
    tps2_181 = tps2_181.drop_duplicates(subset=None, keep='first', inplace=False)
    tps2_181.to_csv(outputfile, encoding='utf-8') #.csv

In [3]:
%%time
for i in range(181,212):
    tps2_181('tps2000000000'+str(i),'tps2_'+str(i)+'.csv')

Wall time: 6h 51min 33s
